In [1]:
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install pandas
!pip install requests
!pip install bs4

In [2]:
import pandas as pd
import time
import requests
import chromedriver_autoinstaller

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [3]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
def click_element(driver, xpath):
    driver.find_element(By.XPATH, xpath).click()

In [5]:
def scrape_page(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela = soup.find('div', {'class': {'col-lg-11 col-md-11 col-sm-12'}})
    titulos = tabela.find_all('strong', {})
    conteudos = tabela.find_all('p')[0::2]
    datas_publicacao = tabela.find_all('p', {'class': 'data-hora'})

    lista_titulos = [titulo.get_text().strip() for titulo in titulos]
    lista_conteudos = [conteudo.get_text().strip() for conteudo in conteudos]
    lista_datas_publicacao = [data.get_text().strip() for data in datas_publicacao]

    return pd.DataFrame([lista_titulos, lista_conteudos, lista_datas_publicacao]).T

In [6]:
service = webdriver.chrome.service.Service()

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)

In [7]:
driver.get("https://anmlegis.datalegis.net/action/ActionDatalegis.php?acao=apresentacao&cod_modulo=351&cod_menu=8014")

In [8]:
# Click em "Títulos Outorgados"
click_element(driver, '//*[@id="column-2"]/div[1]/a[4]')

In [9]:
# Click em "Registro de extração"
click_element(driver, '//*[@id="column-2"]/div/div[2]/div[4]/ul/li[6]/a')

In [10]:
# Click em "2024"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[1]/a')
scroll_to_bottom(driver)
df_extracao2024 = scrape_page(driver)
driver.back()

In [11]:
# Click em "2023"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[2]/a')
scroll_to_bottom(driver)
df_extracao2023 = scrape_page(driver)
driver.back()

In [12]:
# Click em "2022"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[3]/a')
scroll_to_bottom(driver)
df_extracao2022 = scrape_page(driver)
driver.back()

In [13]:
# Click em "2021"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[4]/a')
scroll_to_bottom(driver)
df_extracao2021 = scrape_page(driver)
driver.back()

In [14]:
# Click em "2020"
click_element(driver, '//*[@id="column-2"]/div[1]/ul/li[5]/a')
scroll_to_bottom(driver)
df_extracao2020 = scrape_page(driver)
driver.back()

In [15]:
registroExtracao = pd.concat([df_extracao2020, df_extracao2021, df_extracao2022, df_extracao2023, df_extracao2024])

In [16]:
registroExtracao

,0,1,2
0,REGISTRO DE EXTRAÇÃO Nº 73/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",31/12/2020 | 09:26:08
1,REGISTRO DE EXTRAÇÃO Nº 72/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",31/12/2020 | 09:10:13
2,REGISTRO DE EXTRAÇÃO Nº 173/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",30/12/2020 | 10:51:46
3,REGISTRO DE EXTRAÇÃO Nº 172/2020,"Expedir a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, ...",30/12/2020 | 10:48:50
4,REGISTRO DE EXTRAÇÃO Nº 170/2020,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",30/12/2020 | 12:24:23
...,...,...,...
49,REGISTRO DE EXTRAÇÃO Nº 4/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:25:00
50,REGISTRO DE EXTRAÇÃO Nº 3/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:20:02
51,REGISTRO DE EXTRAÇÃO Nº 2/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:17:56
52,REGISTRO DE EXTRAÇÃO Nº 1/2024,"Expede a DECLARAÇÃO DE REGISTRO DE EXTRAÇÃO, p...",24/01/2024 | 08:12:07
